In [0]:
import numpy as np
from glob import glob
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
nd1 = np.zeros((0,79),dtype=object)
for x in glob('/content/drive/My Drive/Dataset/Data/*.csv'):
    print(x)
    nd1=np.vstack((nd1,pd.read_csv(x)))

/content/drive/My Drive/Dataset/Data/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
/content/drive/My Drive/Dataset/Data/Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv


In [0]:
Dt1=nd1[:,:-1].astype(float)

In [0]:
LNMV1=nd1[~np.isnan(Dt1).any(axis=1),-1]
DtNMV1=Dt1[~np.isnan(Dt1).any(axis=1)]
#Remove Inf values
LNMIV1=LNMV1[~np.isinf(DtNMV1).any(axis=1)]
DtNMIV1=DtNMV1[~np.isinf(DtNMV1).any(axis=1)]

In [0]:

del(DtNMV1)

In [25]:
DtNMIV1.shape

(8247888, 78)

In [0]:

MCDt1=DtNMIV1[LNMIV1 !='Benign',:]

In [28]:
MCDt1.shape

(2170743, 78)

In [0]:
MCL1=LNMIV1[LNMIV1!='Benign']

In [30]:
LNMIV1.shape,MCL1.shape

((8247888,), (2170743,))

In [0]:
np.save('/content/drive/My Drive/Result/NMCx1', MinMaxScaler().fit_transform(MCDt1))
np.save('/content/drive/My Drive/Result/NMCy1', LabelEncoder().fit_transform(MCL1))

In [0]:
nd2 = np.zeros((0,79),dtype=object)
nd2=np.vstack((nd2,pd.read_csv('/content/drive/My Drive/Dataset/Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv')))

In [0]:
Dt2=nd2[:,:-1].astype(float)

In [0]:
LNMV2=nd2[~np.isnan(Dt2).any(axis=1),-1]
DtNMV2=Dt2[~np.isnan(Dt2).any(axis=1)]
LNMIV2=LNMV2[~np.isinf(DtNMV2).any(axis=1)]
DtNMIV2=DtNMV2[~np.isinf(DtNMV2).any(axis=1)]

In [0]:

del(DtNMV2)

In [0]:
DtNMIV2.shape

(7889295, 78)

In [0]:

MCDt2=DtNMIV2[LNMIV2 !='Benign',:]

In [0]:
MCDt2.shape

(576191, 78)

In [0]:
MCL2=LNMIV2[LNMIV2!='Benign']

In [0]:
LNMIV2.shape,MCL2.shape

((7889295,), (576191,))

In [0]:
np.save('/content/drive/My Drive/Result/NMCx2', MinMaxScaler().fit_transform(MCDt2))
np.save('/content/drive/My Drive/Result/NMCy2', LabelEncoder().fit_transform(MCL2))

In [3]:

import numpy as np
import os
import tensorflow as tf
import keras
from keras import models
from keras import layers

Using TensorFlow backend.


In [0]:
x1=np.load('/content/drive/My Drive/DistKeras/NMCx1.npy')
y1=np.load('/content/drive/My Drive/DistKeras/NMCy1.npy')
x2=np.load('/content/drive/My Drive/DistKeras/NMCx2.npy')
y2=np.load('/content/drive/My Drive/DistKeras/NMCy2.npy')
x=np.concatenate((x1,x2), axis=0)
y=np.concatenate((y1,y2), axis=0)
x=x.astype(np.float32)

In [0]:
x=np.delete(x,0,1)

In [6]:
(unique, counts) = np.unique(y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[     0 862382]
 [     1    611]
 [     2    230]
 [     3 686012]
 [     4   1730]
 [     5  41508]
 [     6 461912]
 [     7 139890]
 [     8  10990]
 [     9 193354]
 [    10 160639]
 [    11     87]
 [    12 187589]]


In [9]:
x.shape

(2746934, 77)

In [0]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=123)

In [0]:
from keras.utils import to_categorical
ztrain = to_categorical(ytrain, num_classes=None)
ztest = to_categorical(ytest, num_classes=None)

In [15]:
ztrain.shape

(2197547, 13)

In [0]:
def create_model():
  model = models.Sequential()
  model.add(layers.Dense(256,input_dim=x.shape[1],activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(128,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(64,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(32,activation='relu',use_bias=True))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(np.unique(y).shape[0],activation='softmax',use_bias=True)) 
  return model

from keras.optimizers import * 
model = create_model()
opt = Adam(learning_rate=1e-4)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['categorical_accuracy'])
model.summary()
history=model.fit(xtrain , ztrain ,batch_size=256 , epochs=100 ,verbose=1 ,shuffle=True)
result = model.evaluate(xtest,ztest)
dict(zip(model.metrics_names, result))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 256)               19968     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)               

{'categorical_accuracy': 0.9663370251655579, 'loss': 0.06522966549859952}

In [0]:
pred=model.predict(xtest)
y_pred = np.argmax(pred, axis=-1)

In [0]:
from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, y_pred))
print("The precision score is",metrics.precision_score(ytest, y_pred,average='weighted'))
print("The recall score is",metrics.recall_score(ytest, y_pred,average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, y_pred,average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, y_pred))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, y_pred))
print('\n')

The accuracy score is 0.9663370265404897


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The precision score is 0.9671175309001078
The recall score is 0.9663370265404897
The f1-score is 0.9647334369645175


[[172223      0      0      0      0      0      0      0      0      0
      26      0      0]
 [     6     81      0      0      0      0      0      0      0      0
      47      0      0]
 [     9      0     20      0      0      0      0      0      0      0
      13      0      0]
 [     0      0      0 137180      0      0      0      0      0      0
       0      0      0]
 [     0      0      0      0    333      0      0      0      0      0
       0      0      0]
 [     0      0      0      0      0   8330     68      0      0      0
       0      0      0]
 [     0      0      0      0      0      1  92421      0      0      0
       0      0      0]
 [     0      0      0      0      0      0      0  14088      0  13764
       0      0      0]
 [     0      0      0      0      0      1      0      0   2191      0
      14      0      0]
 [     0      0   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00    172249
           1       0.94      0.60      0.74       134
           2       1.00      0.48      0.65        42
           3       1.00      1.00      1.00    137180
           4       1.00      1.00      1.00       333
           5       1.00      0.99      1.00      8398
           6       1.00      1.00      1.00     92422
           7       0.76      0.51      0.61     27852
           8       1.00      0.99      1.00      2206
           9       0.72      0.89      0.79     38966
          10       1.00      1.00      1.00     32029
          11       0.00      0.00      0.00        17
          12       1.00      1.00      1.00     37559

    accuracy                           0.97    549387
   macro avg       0.88      0.80      0.83    549387
weighted avg       0.97      0.97      0.96    549387





In [0]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc_history=rfc.fit(xtrain,ytrain)
rfc_predict = rfc.predict(xtest)

In [36]:

from sklearn import metrics
print("The accuracy score is",metrics.accuracy_score(ytest, rfc_predict.round()))
print("The precision score is",metrics.precision_score(ytest, rfc_predict.round(),average='weighted'))
print("The recall score is",metrics.recall_score(ytest, rfc_predict.round(),average='weighted'))
print("The f1-score is",metrics.f1_score(ytest, rfc_predict.round(),average='weighted'))
print('\n')
print(metrics.confusion_matrix(ytest, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(metrics.classification_report(ytest, rfc_predict))
print('\n')

The accuracy score is 0.9666064176982709
The precision score is 0.9670922150258797
The recall score is 0.9666064176982709
The f1-score is 0.9652494620412682


[[172240      0      0      0      0      0      0      0      0      0
       9      0      0]
 [     5     89      0      0      0      0      0      0      0      0
      40      0      0]
 [     7      0     29      0      0      0      0      0      0      0
       6      0      0]
 [     0      0      0 137172      0      0      0      0      0      0
       8      0      0]
 [     0      0      0      0    333      0      0      0      0      0
       0      0      0]
 [     0      0      0      0      0   8398      0      0      0      0
       0      0      0]
 [     0      0      0      0      0      0  92422      0      0      0
       0      0      0]
 [     0      0      0      0      0      0      0  14534      0  13318
       0      0      0]
 [     0      0      0      0      0      0      0      0   2183      0
 